In [66]:
# UPDATE TO MATCH PERSONAL DIRECTORY STRUCTURE
DIR = 'C:\\Users\\jesse\\Documents\\GitHub\\vanity_sizing_project\\data\\'

# Opening Files

In [64]:
import gzip
import pandas as pd

In [65]:
# Function to process a chunk of data
def process_data(chunk, columns=None):
    # If columns is not None, keep only those columns
    if columns is not None:
        chunk = chunk[columns]
    return chunk

# Function to read data in chunks and process each chunk
def load_data(file_name, head = None, columns=None, chunksize = 1000):
    chunks = []
    count = 0
    with gzip.open(file_name) as fin:
        for chunk in pd.read_json(fin, lines=True, chunksize=chunksize):
            # Process the chunk
            processed_chunk = process_data(chunk, columns)
            chunks.append(processed_chunk)
            
            count += 1
            # break if reaches the head-th chunk
            if (head is not None) and (count > head):
                break

    # Combine all chunks into a single DataFrame
    df = pd.concat(chunks, ignore_index=True)
    
    return df

In [68]:
rtr = load_data(DIR+'renttherunway_final_data.json.gz')
rtr.head(10)

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28.0,"April 20, 2016"
1,fit,273551,34b,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013"
2,fit,360448,NaN,1063761,NaN,10.0,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,"5' 4""",4,116.0,"December 14, 2015"
3,fit,909926,34c,126335,135lbs,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34.0,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27.0,"September 26, 2016"
5,fit,734848,32b,364092,138lbs,8.0,date,Didn't actually wear it. It fit perfectly. The...,athletic,Traditional with a touch a sass,dress,"5' 8""",8,45.0,"April 30, 2016"
6,fit,336066,34c,568429,112lbs,10.0,everyday,This dress is so sweet. I loved the print. The...,hourglass,LITERALLY THE CUTEST DRESS EVER,dress,"5' 3""",4,27.0,"December 7, 2017"
7,fit,86661,34d+,130259,118lbs,10.0,formal affair,Fit was great. Maybe a little tight under the ...,full bust,"Great dress, beautifully made. I received lot...",dress,"5' 3""",8,65.0,"January 1, 2013"
8,fit,166228,36d,1729232,NaN,10.0,formal affair,I was nervous of it looking cheap when it arri...,full bust,Great for black tie event!,gown,"5' 6""",21,27.0,"June 27, 2016"
9,fit,154309,32b,1729232,114lbs,10.0,formal affair,The dress was very flattering and fit perfectl...,petite,This dress was everything! It was perfect for ...,gown,"5' 3""",1,33.0,"October 17, 2016"


# Rent The Runway

## General Overview

In [69]:
rtr.describe()

,user_id,item_id,rating,size,age
count,192544.000000,1.925440e+05,192462.000000,192544.000000,191584.000000
mean,499494.100149,1.045684e+06,9.092371,12.245175,33.871017
std,289059.719328,8.053148e+05,1.430044,8.494877,8.058083
min,9.000000,1.233730e+05,2.000000,0.000000,0.000000
25%,250654.250000,1.950760e+05,8.000000,8.000000,29.000000
50%,499419.000000,9.483960e+05,10.000000,12.000000,32.000000
75%,750974.000000,1.678888e+06,10.000000,16.000000,37.000000
max,999997.000000,2.966087e+06,10.000000,58.000000,117.000000


In [70]:
rtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192544 entries, 0 to 192543
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   fit             192544 non-null  object 
 1   user_id         192544 non-null  int64  
 2   bust size       174133 non-null  object 
 3   item_id         192544 non-null  int64  
 4   weight          162562 non-null  object 
 5   rating          192462 non-null  float64
 6   rented for      192534 non-null  object 
 7   review_text     192544 non-null  object 
 8   body type       177907 non-null  object 
 9   review_summary  192544 non-null  object 
 10  category        192544 non-null  object 
 11  height          191867 non-null  object 
 12  size            192544 non-null  int64  
 13  age             191584 non-null  float64
 14  review_date     192544 non-null  object 
dtypes: float64(2), int64(3), object(10)
memory usage: 22.0+ MB


In [71]:
#what is the most commmon weight - height - bust size combination
# Create a new column that combines weight, height, and bust size
rtr['summary'] = rtr['weight'].astype(str) + '-' + rtr['height'].astype(str) + '-' + rtr['bust size'].astype(str) + '-' + rtr['body type'].astype(str) + '-' + rtr['age'].astype(str)

# Find the most common combination excluding when weight, height, or bust size is missing
rtr['summary'].value_counts().head(10)

summary
125lbs-5' 3"-34c-full bust-31.0            437
122lbs-5' 7"-34b-straight & narrow-26.0    293
nan-5' 5"-34ddd/e-full bust-23.0           228
130lbs-5' 7"-34b-pear-26.0                 145
130lbs-5' 9"-34c-athletic-34.0             133
110lbs-5' 6"-32a-straight & narrow-24.0    124
125lbs-5' 2"-34d-full bust-32.0            105
nan-5' 4"-34b-hourglass-26.0               103
135lbs-5' 6"-34dd-hourglass-45.0            96
nan-5' 4"-nan-nan-23.0                      87
Name: count, dtype: int64

## General Cleaning

### Dropping duplicates

In [72]:
print(rtr.duplicated().sum())
rtr = rtr.drop_duplicates()

189


### Changing weight to numercial value

In [73]:
print(rtr['weight'].head(10))

0    137lbs
1    132lbs
2       NaN
3    135lbs
4    145lbs
5    138lbs
6    112lbs
7    118lbs
8       NaN
9    114lbs
Name: weight, dtype: object


In [74]:
rtr['weight'] = rtr['weight'].str.replace('lbs', '')
rtr['weight'] = pd.to_numeric(rtr['weight'], errors='raise')

### Changing height to numerical value -- to inches

In [75]:
print(rtr['height'].head(10))

0    5' 8"
1    5' 6"
2    5' 4"
3    5' 5"
4    5' 9"
5    5' 8"
6    5' 3"
7    5' 3"
8    5' 6"
9    5' 3"
Name: height, dtype: object


In [76]:
def convert_to_inches(height):
    height_parts = height.split("'")
    feet = int(height_parts[0].strip())
    inches = int(height_parts[1].strip().replace('"', ''))
    total_inches = (feet * 12) + inches
    return total_inches

In [77]:
rtr.loc[rtr['height'].notnull(), 'height'] = rtr.loc[rtr['height'].notnull(), 'height'].apply(convert_to_inches)
rtr['height'].head(10)
rtr['height'] = pd.to_numeric(rtr['height'], errors='coerce').astype('Int64')

### Restructuring Bust Size

In [78]:
rtr['bust size'].value_counts()

bust size
34b    27255
34c    23117
34d    18013
36c    13502
32d    11188
       ...  
28h        1
46f        1
42j        1
28i        1
44h        1
Name: count, Length: 106, dtype: int64

In [79]:
#split the bust size into band and cup
rtr['band_size'] = rtr['bust size'].str.extract('(\d+)').astype(float)
rtr['band_size'].value_counts()
rtr['band_size'] = pd.to_numeric(rtr['band_size'], errors='coerce').astype('Int64')

In [80]:
rtr['band_size'] = pd.to_numeric(rtr['band_size'], errors='coerce').astype('Int64')

In [81]:
rtr['cup_size'] = rtr['bust size'].str.extract('([A-Za-z]+\+?)')
rtr['cup_size'].value_counts()

cup_size
c      49586
b      49067
d      44967
a      15271
dd      6690
d+      3950
ddd     2598
aa       710
f        458
g        440
h        156
i         40
j         30
Name: count, dtype: int64

In [82]:
rtr = rtr.drop(columns='bust size')

### Changing data types where necessary

In [83]:
rtr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192355 entries, 0 to 192543
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   fit             192355 non-null  object 
 1   user_id         192355 non-null  int64  
 2   item_id         192355 non-null  int64  
 3   weight          162400 non-null  float64
 4   rating          192274 non-null  float64
 5   rented for      192345 non-null  object 
 6   review_text     192355 non-null  object 
 7   body type       177730 non-null  object 
 8   review_summary  192355 non-null  object 
 9   category        192355 non-null  object 
 10  height          191680 non-null  Int64  
 11  size            192355 non-null  int64  
 12  age             191395 non-null  float64
 13  review_date     192355 non-null  object 
 14  summary         192355 non-null  object 
 15  band_size       173963 non-null  Int64  
 16  cup_size        173963 non-null  object 
dtypes: Int64(2), fl

In [84]:
rtr['review_date'] = pd.to_datetime(rtr['review_date'])
rtr['review_date'].dt.year.value_counts()

review_date
2017    67798
2016    51298
2015    30550
2014    28266
2013     9617
2012     3362
2018     1254
2011      209
2010        1
Name: count, dtype: int64

In [85]:
rtr['age'] = pd.to_numeric(rtr['age'], errors='coerce').astype('Int64')

### Category coding with consolidation

In [86]:
def to_category_w_dict(df, column):
    df[column] = df[column].astype('category')
    category_dict = {category: code for code, category in enumerate(df[column].cat.categories)}
    return category_dict

In [87]:
#print count for each 'rented for'
rtr['rented for'].value_counts()

rented for
wedding            57737
formal affair      40380
party              35578
everyday           16810
other              15366
work               15029
date                7382
vacation            4062
party: cocktail        1
Name: count, dtype: int64

In [88]:
#rewrite party: cocktail to just party
rtr['rented for'] = rtr['rented for'].str.replace('party: cocktail', 'party')

In [89]:
rtr['category'].value_counts().head(50)

category
dress         92752
gown          44377
sheath        19264
shift          5364
jumpsuit       5184
top            4931
maxi           3443
romper         3070
jacket         2404
mini           1751
skirt          1531
sweater        1149
coat            980
blazer          782
shirtdress      729
blouse          651
down            464
pants           422
vest            278
shirt           277
cardigan        241
frock           205
culottes        188
tank            181
tunic           162
bomber          128
sweatshirt      125
suit            123
leggings        112
pant            107
print           103
legging          87
cape             84
culotte          75
pullover         58
midi             57
trouser          55
poncho           48
knit             48
peacoat          39
turtleneck       34
kimono           30
tee              22
trench           20
trousers         18
kaftan           17
parka            17
cami             16
ballgown         16
tight      

In [90]:
#export category value counts to excel
#rtr['category'].value_counts().to_excel(DIR+'category_value_counts.xlsx')
new_cats = pd.read_excel(DIR+'category_value_counts.xlsx')
new_cats

,category,count,Category 1,Category 2
0,dress,92752,dress,dress
1,gown,44377,dress,gown
2,sheath,19264,dress,sheath_dress
3,shift,5364,dress,shift_dress
4,maxi,3443,dress,maxi_dress
...,...,...,...,...
63,cami,16,top,tank
64,blouson,14,top,blouse
65,t-shirt,13,top,top
66,henley,8,top,top


In [91]:
#merge new_cats with rtr, matching on category and only pulling in Category 1 and Category 2
rtr = rtr.merge(new_cats, on='category', how='left')
rtr = rtr.drop(columns='category')
rtr = rtr.drop(columns='count')
rtr = rtr.rename(columns={'Category 1': 'category', 'Category 2': 'subcategory'})

In [92]:
#apply the function to the category columns, and store the dictionaries
category_dicts = {}
for column in ['fit','rating','rented for', 'body type','category','subcategory','size','cup_size','band_size']:
    category_dicts[column] = to_category_w_dict(rtr, column)

In [93]:
category_dicts

{'fit': {'fit': 0, 'large': 1, 'small': 2},
 'rating': {2.0: 0, 4.0: 1, 6.0: 2, 8.0: 3, 10.0: 4},
 'rented for': {'date': 0,
  'everyday': 1,
  'formal affair': 2,
  'other': 3,
  'party': 4,
  'vacation': 5,
  'wedding': 6,
  'work': 7},
 'body type': {'apple': 0,
  'athletic': 1,
  'full bust': 2,
  'hourglass': 3,
  'pear': 4,
  'petite': 5,
  'straight & narrow': 6},
 'category': {'dress': 0,
  'jumpsuit': 1,
  'other': 2,
  'outerwear': 3,
  'pants': 4,
  'skirt': 5,
  'top': 6},
 'subcategory': {'blazer': 0,
  'blouse': 1,
  'coat': 2,
  'dress': 3,
  'gown': 4,
  'jacket': 5,
  'jumpsuit': 6,
  'leggings': 7,
  'maxi_dress': 8,
  'mini_dress': 9,
  'other': 10,
  'other_dress': 11,
  'other_outer': 12,
  'other_pants': 13,
  'pant': 14,
  'pants': 15,
  'romper': 16,
  'sheath_dress': 17,
  'shift_dress': 18,
  'shirt': 19,
  'shorts': 20,
  'skirt': 21,
  'suit': 22,
  'sweater': 23,
  'sweatshirt': 24,
  'tank': 25,
  'top': 26},
 'size': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,

#### Review data cleaning

In [94]:
rtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192355 entries, 0 to 192354
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fit             192355 non-null  category      
 1   user_id         192355 non-null  int64         
 2   item_id         192355 non-null  int64         
 3   weight          162400 non-null  float64       
 4   rating          192274 non-null  category      
 5   rented for      192345 non-null  category      
 6   review_text     192355 non-null  object        
 7   body type       177730 non-null  category      
 8   review_summary  192355 non-null  object        
 9   height          191680 non-null  Int64         
 10  size            192355 non-null  category      
 11  age             191395 non-null  Int64         
 12  review_date     192355 non-null  datetime64[ns]
 13  summary         192355 non-null  object        
 14  band_size       173963 non-null  cat

In [99]:
category_dicts

{'fit': {'fit': 0, 'large': 1, 'small': 2},
 'rating': {2.0: 0, 4.0: 1, 6.0: 2, 8.0: 3, 10.0: 4},
 'rented for': {'date': 0,
  'everyday': 1,
  'formal affair': 2,
  'other': 3,
  'party': 4,
  'vacation': 5,
  'wedding': 6,
  'work': 7},
 'body type': {'apple': 0,
  'athletic': 1,
  'full bust': 2,
  'hourglass': 3,
  'pear': 4,
  'petite': 5,
  'straight & narrow': 6},
 'category': {'dress': 0,
  'jumpsuit': 1,
  'other': 2,
  'outerwear': 3,
  'pants': 4,
  'skirt': 5,
  'top': 6},
 'subcategory': {'blazer': 0,
  'blouse': 1,
  'coat': 2,
  'dress': 3,
  'gown': 4,
  'jacket': 5,
  'jumpsuit': 6,
  'leggings': 7,
  'maxi_dress': 8,
  'mini_dress': 9,
  'other': 10,
  'other_dress': 11,
  'other_outer': 12,
  'other_pants': 13,
  'pant': 14,
  'pants': 15,
  'romper': 16,
  'sheath_dress': 17,
  'shift_dress': 18,
  'shirt': 19,
  'shorts': 20,
  'skirt': 21,
  'suit': 22,
  'sweater': 23,
  'sweatshirt': 24,
  'tank': 25,
  'top': 26},
 'size': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,

In [100]:
rtr.to_pickle('rtr_reviews_uncleaned.pkl')

import json
# Convert float and int64 keys to int
category_dicts['rating'] = {int(key): value for key, value in category_dicts['rating'].items()}
category_dicts['size'] = {int(key): value for key, value in category_dicts['size'].items()}
category_dicts['band_size'] = {int(key): value for key, value in category_dicts['band_size'].items()}

# Now you can export the dictionary
with open('category_dict.json', 'w') as f:
    json.dump(category_dicts, f)